# अध्याय ७: च्याट एप्लिकेसनहरू निर्माण गर्दै
## Github मोडेल्स API छिटो सुरु गर्ने

यो नोटबुक [Azure OpenAI Samples Repository](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst) बाट अनुकूलित गरिएको हो जसमा [Azure OpenAI](notebook-azure-openai.ipynb) सेवाहरूमा पहुँच हुने नोटबुकहरू समावेश छन्।


# अवलोकन  
"ठूला भाषा मोडेलहरू पाठलाई पाठमा रूपान्तरण गर्ने कार्यहरू हुन्। कुनै इनपुट स्ट्रिङ दिँदा, ठूला भाषा मोडेलले त्यसपछि आउने पाठको अनुमान गर्न खोज्छ"(1)। यो "छिटो सुरु" नोटबुकले प्रयोगकर्ताहरूलाई उच्च-स्तरका LLM अवधारणाहरू, AML सँग सुरु गर्न आवश्यक मुख्य प्याकेजहरू, प्रॉम्प्ट डिजाइनको हल्का परिचय, र विभिन्न प्रयोगका लागि केही छोटा उदाहरणहरू प्रस्तुत गर्नेछ।


## सामग्री सूची  

[सारांश](../../../../07-building-chat-applications/python)  
[OpenAI सेवा कसरी प्रयोग गर्ने](../../../../07-building-chat-applications/python)  
[1. तपाईंको OpenAI सेवा सिर्जना गर्दै](../../../../07-building-chat-applications/python)  
[2. स्थापना](../../../../07-building-chat-applications/python)    
[3. प्रमाणिकरण विवरणहरू](../../../../07-building-chat-applications/python)  

[प्रयोगका क्षेत्रहरू](../../../../07-building-chat-applications/python)    
[1. पाठ संक्षेप गर्नुहोस्](../../../../07-building-chat-applications/python)  
[2. पाठ वर्गीकरण गर्नुहोस्](../../../../07-building-chat-applications/python)  
[3. नयाँ उत्पादन नामहरू सिर्जना गर्नुहोस्](../../../../07-building-chat-applications/python)  
[4. वर्गीकरणकर्ता फाइन ट्युन गर्नुहोस्](../../../../07-building-chat-applications/python)  

[सन्दर्भहरू](../../../../07-building-chat-applications/python)


### आफ्नो पहिलो प्रम्प्ट बनाउनुहोस्  
यो छोटो अभ्यासले Github Models मा मोडेललाई प्रम्प्ट पठाउने आधारभूत परिचय दिनेछ, जहाँ तपाईंले "सारांश" जस्तो साधारण कार्य गर्नुहुन्छ।

**चरणहरू**:  
1. यदि तपाईंले अझै नगर्नुभएको छ भने, आफ्नो python वातावरणमा `azure-ai-inference` लाइब्रेरी इन्स्टल गर्नुहोस्।  
2. स्ट्यान्डर्ड सहायक लाइब्रेरीहरू लोड गर्नुहोस् र Github Models को लागि प्रमाणिकरण सेटअप गर्नुहोस्।  
3. आफ्नो कार्यका लागि उपयुक्त मोडेल छान्नुहोस्  
4. मोडेलका लागि साधारण प्रम्प्ट तयार गर्नुहोस्  
5. आफ्नो अनुरोध मोडेल API मा पठाउनुहोस्!


### 1. `azure-ai-inference` स्थापना गर्नुहोस्


In [ ]:
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

### ३. सही मोडेल फेला पार्नुहोस्  
GPT-3.5-turbo वा GPT-4 मोडेलहरूले प्राकृतिक भाषा बुझ्न र उत्पन्न गर्न सक्छन्।


In [ ]:
# Select the General Purpose curie model for text
model_name = "gpt-4o"

## ४. प्रम्प्ट डिजाइन  

"ठूला भाषा मोडेलहरूको जादु भनेको, ठूलो मात्रामा पाठमा भविष्यवाणी त्रुटि घटाउने तालिम गर्दा, मोडेलहरूले ती भविष्यवाणीका लागि उपयोगी अवधारणाहरू सिक्छन्। उदाहरणका लागि, उनीहरूले यस्ता अवधारणाहरू सिक्छन्"(१):

* कसरी शब्द लेख्ने
* व्याकरण कसरी काम गर्छ
* कसरी पुनर्लेखन गर्ने
* कसरी प्रश्नको उत्तर दिने
* कसरी संवाद गर्ने
* धेरै भाषामा लेख्ने तरिका
* कसरी कोड लेख्ने
* आदि।

#### ठूला भाषा मोडेललाई कसरी नियन्त्रण गर्ने  
"ठूला भाषा मोडेलमा सबैभन्दा प्रभावशाली इनपुट भनेको पाठ प्रम्प्ट हो(१)।

ठूला भाषा मोडेलहरूलाई केही तरिकाले आउटपुट दिन प्रेरित गर्न सकिन्छ:

निर्देशन: मोडेललाई के चाहिएको हो भनेर बताउनुहोस्
पूरा गर्नु: तपाईंलाई चाहिएको कुरा सुरु गरेर मोडेललाई पूरा गर्न लगाउनुहोस्
प्रदर्शन: मोडेललाई के चाहिएको हो देखाउनुहोस्, या त:
प्रम्प्टमा केही उदाहरणहरू
फाइन-ट्युनिङ तालिम डेटासेटमा सयौं वा हजारौं उदाहरणहरू"



#### प्रम्प्ट बनाउँदा तीन आधारभूत दिशानिर्देशहरू छन्:

**देखाउनुहोस् र बताउनुहोस्**। तपाईंलाई के चाहिएको हो स्पष्ट बनाउनुहोस्, या त निर्देशन, उदाहरण, वा दुवैको संयोजनबाट। यदि तपाईं मोडेललाई वस्तुहरूको सूची वर्णानुक्रममा क्रमबद्ध गर्न वा अनुच्छेदलाई भावनाद्वारा वर्गीकृत गर्न चाहनुहुन्छ भने, उसलाई त्यही चाहिएको हो भनेर देखाउनुहोस्।

**गुणस्तरीय डेटा दिनुहोस्**। यदि तपाईं वर्गीकरण बनाउने प्रयास गर्दै हुनुहुन्छ वा मोडेललाई कुनै ढाँचा पछ्याउन लगाउन चाहनुहुन्छ भने, पर्याप्त उदाहरणहरू छन् कि छैनन् हेर्नुहोस्। तपाईंका उदाहरणहरू राम्रोसँग जाँच गर्नुहोस् — मोडेल सामान्यतया आधारभूत हिज्जे त्रुटिहरू छुट्याउन सक्नेछ र तपाईंलाई उत्तर दिनेछ, तर उसले यो जानाजानी गरिएको हो भनेर पनि मान्न सक्छ र यसले उत्तरमा असर गर्न सक्छ।

**सेटिङहरू जाँच गर्नुहोस्।** तापक्रम र top_p सेटिङहरूले मोडेलले उत्तर दिने क्रममा कति निश्चित छ भन्ने नियन्त्रण गर्छ। यदि तपाईंले एउटा मात्र सही उत्तर चाहनुहुन्छ भने, यी सेटिङहरू कम राख्नुहोस्। यदि तपाईं विविध उत्तरहरू खोज्दै हुनुहुन्छ भने, यी उच्च राख्न सक्नुहुन्छ। मानिसहरूले यी सेटिङहरूसँग गर्ने सबैभन्दा ठूलो गल्ती भनेको यी "बुद्धिमत्ता" वा "रचनात्मकता" नियन्त्रण हुन् भनेर सोच्नु हो।


स्रोत: https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## पाठ संक्षेप गर्नुहोस्  
#### चुनौती  
पाठको अन्त्यमा 'tl;dr:' थपेर पाठलाई संक्षेप गर्नुहोस्। मोडेलले कुनै अतिरिक्त निर्देशन बिना नै धेरै कार्यहरू कसरी गर्न सक्छ भन्ने कुरा ध्यान दिनुहोस्। तपाईंले tl;dr भन्दा बढी वर्णनात्मक प्रॉम्प्टहरू प्रयोग गरेर मोडेलको व्यवहार परिवर्तन गर्न र आफूले चाहेको संक्षेप अनुकूल बनाउन प्रयोग गर्न सक्नुहुन्छ(3)।  

हालैको अनुसन्धानले देखाएको छ कि ठूलो पाठ संग्रहमा प्रि-ट्रेनिङ गरेर र त्यसपछि विशेष कार्यमा फाइन-ट्युनिङ गर्दा धेरै NLP कार्यहरू र बेन्चमार्कहरूमा उल्लेखनीय प्रगति भएको छ। यद्यपि यो विधि सामान्यतया कार्य-निरपेक्ष संरचनामा आधारित हुन्छ, तर अझै पनि यसमा हजारौं वा दशौं हजार उदाहरणहरूको कार्य-विशिष्ट फाइन-ट्युनिङ डाटासेट आवश्यक पर्छ। यसको विपरीत, मानिसहरूले सामान्यतया केही उदाहरणहरू वा साधारण निर्देशनहरूबाट नयाँ भाषा कार्य गर्न सक्छन् - जुन अहिलेका NLP प्रणालीहरूले अझै पनि गर्न गाह्रो मान्छन्। यहाँ हामी देखाउँछौं कि भाषा मोडेलको आकार बढाउँदा कार्य-निरपेक्ष, थोरै उदाहरणमा आधारित प्रदर्शनमा धेरै सुधार आउँछ, कहिलेकाहीँ त अघिल्लो उत्कृष्ट फाइन-ट्युनिङ विधिहरूसँग प्रतिस्पर्धा गर्न सक्ने स्तरमा पुग्छ।



Tl;dr


# विभिन्न प्रयोगका लागि अभ्यासहरू  
1. पाठ संक्षेप गर्नुहोस्  
2. पाठ वर्गीकरण गर्नुहोस्  
3. नयाँ उत्पादन नामहरू सिर्जना गर्नुहोस्


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## पाठ वर्गीकरण  
#### चुनौती  
वर्गीकरणका लागि वस्तुहरूलाई इन्फरेन्स समयमा दिइएका श्रेणीहरूमा वर्गीकृत गर्नुहोस्। तलको उदाहरणमा, हामीले श्रेणीहरू र वर्गीकरण गर्नुपर्ने पाठ दुवै प्रॉम्प्टमा दिएका छौं (*playground_reference)।

ग्राहकको सोधपुछ: नमस्कार, मेरो ल्यापटपको किबोर्डको एउटा कुञ्जी भर्खरै बिग्रिएको छ र मलाई नयाँ चाहिन्छ:

वर्गीकृत श्रेणी:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## नयाँ उत्पादन नामहरू सिर्जना गर्नुहोस्
#### चुनौती
उदाहरण शब्दहरूबाट उत्पादन नामहरू सिर्जना गर्नुहोस्। यहाँ हामीले उत्पादनको बारेमा जानकारी समावेश गरेका छौं जसका लागि हामी नामहरू सिर्जना गर्दैछौं। हामीले चाहेको ढाँचा देखाउनका लागि यस्तै एउटा उदाहरण पनि दिएका छौं। थप रचनात्मक र अनौठो उत्तरहरूका लागि हामीले temperature मान उच्च राखेका छौं।

उत्पादन विवरण: घरमै बनाउने मिल्कशेक मेकर
बीउ शब्दहरू: छिटो, स्वस्थ, सानो।
उत्पादन नामहरू: HomeShaker, Fit Shaker, QuickShake, Shake Maker

उत्पादन विवरण: जुनसुकै खुट्टाको साइजमा मिल्ने जुत्ताको जोडी।
बीउ शब्दहरू: अनुकूल, फिट, omni-fit.


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# सन्दर्भहरू  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [OpenAI Studio Examples](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [GPT-3 लाई पाठ वर्गीकरणका लागि फाइन-ट्युन गर्नका लागि उत्कृष्ट अभ्यासहरू](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# थप सहायता को लागि  
[OpenAI व्यावसायिकरण टोली](AzureOpenAITeam@microsoft.com)


# योगदानकर्ताहरू
* [Chew-Yean Yam](https://www.linkedin.com/in/cyyam/)



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी शुद्धताको लागि प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटिहरू वा अशुद्धताहरू हुन सक्छन्। मूल भाषा मा रहेको मूल दस्तावेज़लाई नै आधिकारिक स्रोत मान्नुपर्छ। महत्वपूर्ण जानकारीको लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याका लागि हामी जिम्मेवार हुने छैनौं।
